* Open data
* Normalize
* turn to vectors
* dimensionality reduction
* kNN or DTree classifiers

In [1]:
# Open data

import gensim
from gensim.models import KeyedVectors

en_vectors_file = 'numberbatch-en.txt'
uk_vectors_file = 'news.lowercased.tokenized.word2vec.300d'

uk_vectors = KeyedVectors.load_word2vec_format(uk_vectors_file, binary=False)

In [2]:
uk_vectors.most_similar('автомобіль', topn=10)

[('позашляховик', 0.8455106019973755),
 ('джип', 0.8443582057952881),
 ('мотоцикл', 0.8110474348068237),
 ('мікроавтобус', 0.7966219782829285),
 ('автомобіль**', 0.784403383731842),
 ('бус', 0.7747742533683777),
 ('легковик', 0.7658803462982178),
 ('скутер', 0.7596759796142578),
 ('фургон', 0.7572780847549438),
 ('мопед', 0.7557206749916077)]

In [4]:
# Try mathematically subtracting vectors
import numpy as np

def subtract(w1, w2):
    global uk_vectors
    return np.subtract(uk_vectors[w1], uk_vectors[w2])

uk_vectors.similar_by_vector(subtract('ющенко', 'президент'))

[('ющенка', 0.3845677375793457),
 ('балоги', 0.37590137124061584),
 ('ющенко', 0.3565678596496582),
 ('балога', 0.3410945236682892),
 ('януковича', 0.33086124062538147),
 ('пилипишина', 0.32630905508995056),
 ('палинський', 0.3042936325073242),
 ('пинзеника', 0.30348095297813416),
 ('пилипишин', 0.3004416227340698),
 ('янукович', 0.2991790771484375)]

In [3]:
# Note that the `most_similar` methor uses a similar, but not identical implementation
print(uk_vectors.most_similar(positive=['ющенко'], negative=['президент']))

[('ющенка', 0.4185105860233307), ('балога', 0.4034050703048706), ('балоги', 0.3902004361152649), ('янукович', 0.3754698634147644), ('януковича', 0.3654754161834717), ('пилипишин', 0.3459473252296448), ('пинзеник', 0.3352230191230774), ('пилипишина', 0.33186835050582886), ('палинський', 0.3314499855041504), ('гвоздь', 0.3293236494064331)]


In [6]:
from os import path, walk
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from langdetect import detect

from pprint import pprint as pp


import operator
import tokenize_uk
import re


def open_data(root):
    """Open all files in specified root folder and return a list of separated posts"""
    
    dataset = []
    
    # Walk text files in data folder
    for root, dirs, files in walk(top=root):
        for fname in files:
            p = path.join(root, fname)
            
            # Open each file
            with open(p, 'r') as f:
                post_map = {}
                content = ''
                for line in f.readlines():
                    content = content + line
                
                # Split files into individual posts
                posts = [p for p in content.split('\n\n\n') if p != '']
                dataset.extend(posts)
    
    # Filter only Ukrainian posts. Also filter out posts with no features
    try:
        dataset = [p for p in dataset if detect(p) == 'uk']
    except:
        pass
    
    return dataset

def vectorize(post, uk_vectors=uk_vectors):
    """Obtain vector representation of a post by: 
    summing all vectors for each semantically significant word in the post;
    and averaging by number of posts."""

    def normalize(s):
        """A primitive normalization routine"""

        s = s.lower()

        punct = set('.,-*/:;')

        for c in punct:
            s = s.replace(c, '')

        return s
    
    def load_stopwords():
        """Load stopwords from an included list"""

        stopwords = set()
        with open('stopwords.txt', 'r') as f:
            for line in f.readlines():
                stopwords.add(line.strip())

        return stopwords
    
    # Normalize (mostly punctuation)
    post = normalize(post)
        
    # Tokenize and remove stopwords
    words = [w for w in tokenize_uk.tokenize_words(post) if w not in load_stopwords()]
    
    # Init empty vector
    vec = np.zeros(300)
    
    # Sum and average vectors for tokens
    c = 0
    for word in words:
        try:
            vec = np.add(vec, uk_vectors[word])
            c += 1
        except KeyError:
            pass
    
    if c != 0:
        vec = np.array([x / c for x in vec])
    
    return vec
        
def build_map(data):
    """Create a vector map of {(vector): 'post'} to look up posts by vector"""
    
    vec_map = {}
    
    for post in data:
        vec_map[tuple(vectorize(post))] = post
    
    return vec_map

def find_similar(s, vec_map, n=10):
    """Basic cosine similarity clustering algorithm"""
    
    vectors = [k for k in vec_map.keys()]
    target = vectorize(s)
    
    sim_pairs = []
    for i in range(0, len(vectors)):
        a = np.array(vectors[i])
        b = np.array(target)
        
        # Calculate cosine similarity
        dot = np.dot(a, b)
        norma = np.linalg.norm(a)
        normb = np.linalg.norm(b)
        cos = dot / (norma * normb)
        
        sim_pairs.append((vec_map[vectors[i]], cos))
    
    return sorted(sim_pairs, key=lambda x: x[1], reverse=True)[0:n+1]
    

def dbscan(vec_map):
    """Cluster posts in the dataset using the DBSCAN algorithm"""
    
    # Vectors act as keys in the vector map, so let's get them
    vectors = [k for k in vec_map.keys()]
    
    db = DBSCAN(eps=0.5, n_jobs=-1, metric='cosine').fit(vectors)
    labels = db.labels_
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    print(f"Estimated {n_clusters} clusters")
    
    return db

    
root = './1551'

dataset = open_data(root)
vec_map = build_map(dataset)
# db = dbscan(vec_map)


# s = """нічим дихати"""

# pairs = find_similar(s, vec_map)

KeyboardInterrupt: 

In [192]:
pp(pairs)

[('В будинку Л.Гавро 5а особливо зі сторони озера другу ніч і день неможливо '
  'відкрити вікно, запах смогу, що це за запах, не відомо, як нам дихати? '
  'Перевірте будьласка, чи можна дихати цим повітрям та надайте інформацію '
  'жителям!',
  0.4697315548502146),
 ('Прошу зробити капітальний ремон дороги в цьому дворі! Тут не те що машиною '
  'проїхати складно, тут навіть пройти вечором страшно, ноги можна по '
  'виламувати! Дякую',
  0.46093911883301375),
 ('НЕ ПРИБРАНА ДВОРОВАЯ ТЕРИТОРИЯ , ГДЕ ДВОРНИКИ , НЕ ВИДНО',
  0.43826666223715477),
 ('Додаток до мого звернення # А-8346 від 18.09.15.\n'
  'Скажіть, будь-ласка, на фото, що прикрепила, це дійсно норма тиску? І в цей '
  'же час ГОРЯЧА ВОДА просто відсутня! нема нормального напору! Я зверталася з '
  'більшою проблемою поганого тиску ГОРЯЧОЇ ВОДИ! Бо дуже неприємно, коли ти '
  'почав приймати душ, а тут вода зникає і  стоїш намиленою у ванні і чекаєш, '
  'коли з&#039;явиться вода.',
  0.4289672979481472),
 ('Добрий день!\